# Dino Dynasty OS - Phi4-mini Fine-Tuning

Fine-tune Phi4-mini on our custom data!

## Setup

In [ ]:
# Install required packages
!pip install -q transformers>=4.46 accelerate peft datasets bitsandbytes

In [ ]:
# Clone our training data from GitHub
!git clone https://github.com/RekitRex21/Dino-Training.git
import os
os.chdir('Dino-Training')
print('Changed to Dino-Training folder')


In [ ]:
# Check GPU
import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Load & Prepare Data

In [ ]:
import json
import os

# Load training data
data = []

for f in ['rex_assistant.jsonl', 'security_analysis.json', 'trading.json']:
    path = f'{f}'
    if os.path.exists(path):
        with open(path) as fp:
            if f.endswith('.jsonl'):
                data.extend([json.loads(l) for l in fp])
            else:
                data.extend(json.load(fp))

print(f"Loaded {len(data)} training examples")
print(f"Example: {data[0]}")

## Format for Fine-Tuning

In [ ]:
# Format for instruction tuning
def format_instruction(example):
    return f"""### Instruction
{example['instruction']}

### Input
{example.get('input', '')}

### Response
{example['output']}"""

formatted_data = [{"text": format_instruction(d)} for d in data]
print(f"Formatted {len(formatted_data)} examples")
print(formatted_data[0]["text"][:500])

## Load Base Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset

model_name = "microsoft/Phi-4-mini-instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Load model in 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)
print("Model loaded!")

## Setup LoRA

In [ ]:
# LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

## Tokenize & Train

In [ ]:
# Create dataset
dataset = Dataset.from_list(formatted_data)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Training args
training_args = TrainingArguments(
    output_dir="./rex-dino-model",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="epoch",
    save_total_limit=1,
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Train!
trainer.train()
print("Training complete!")

## Export Model

In [ ]:
# Merge and save
merged_model = model.merge_and_unload()
merged_model.save_pretrained("./rex-dino-v1")
tokenizer.save_pretrained("./rex-dino-v1")

# Convert to GGUF for Ollama
!pip install -q llama.cpp
!python -m llama.cpp.quantize ./rex-dino-v1 ./rex-dino-v1-q4.gguf q4_0

print("Model saved as rex-dino-v1-q4.gguf!")
print("Download this file to run on your Dell!")